In [1]:
import pandas as pd
import numpy as np

In [7]:
# Load the data
data = pd.read_csv('/Users/macbook/Documents/PhD_Documents/embedding_methods/embeddings/Llama/llama_mean_embeddings/results/Llama_mean_embedding.csv')

# Drop the unnamed column if it exists
#data = data.drop('Unnamed: 0', axis=1)

# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort the DataFrame by the 'Date' column
data = data.sort_values(by='Date')
data = data.drop('Unnamed: 0', axis = 1)
# Display the sorted DataFrame
print(data)

           Date  embedding_0  embedding_1  embedding_2  embedding_3  \
0    2014-01-01    -1.517078    -2.217229    -0.913382    -0.507521   
1    2014-01-02    -0.979689    -2.329813     0.262478     0.485761   
2    2014-01-03    -0.997662    -1.799282     0.674190     0.402943   
3    2014-01-04    -2.667392    -3.183720     0.446297     2.351726   
4    2014-01-05    -0.737085    -2.182177    -0.897072     0.597660   
...         ...          ...          ...          ...          ...   
3678 2024-01-27    -0.664020    -2.571041     0.326696     0.509027   
3679 2024-01-28    -0.673623    -1.321869     0.994243     0.413169   
3680 2024-01-29    -0.891116    -2.005443     0.658899     0.488497   
3681 2024-01-30    -0.635582    -1.944958     0.485915     0.336782   
3682 2024-01-31    -0.719049    -2.379355     0.390316     0.350448   

      embedding_4  embedding_5  embedding_6  embedding_7  embedding_8  ...  \
0       -0.611082     0.249111    -2.486910    -0.126034    -3.137839

In [8]:
# Load data from CSV file
df = pd.read_csv('/Users/macbook/Documents/PhD_Documents/ECAI/Financial_data/cb-5m.csv',
                 delimiter=';', names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])

# Convert 'Date' and 'Time' columns to datetime format
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df['Date'] = df['Datetime'].dt.date  # Keep only the date part

# Calculate returns (log returns)
df['Return'] = np.log(df['Close'] / df['Close'].shift(1))

# Scale returns by 100
df['Return'] *= 40

# Calculate realized volatility (RV)
df['RV'] = df['Return']**2
realized_volatility = df.groupby('Date')['RV'].sum().reset_index()
realized_volatility.columns = ['Date', 'RV']

# Calculate bipower variation (BVP)
df['Abs_Return'] = df['Return'].abs()
df['BVP'] = df['Abs_Return'].shift(1) * df['Abs_Return']
bipower_variation = df.groupby('Date')['BVP'].sum().reset_index()
bipower_variation.columns = ['Date', 'BVP']

# Calculate negative and positive realized volatility (RV_neg, RV_pos)
df['RV_neg'] = np.where(df['Return'] < 0, df['Return']**2, 0)
df['RV_pos'] = np.where(df['Return'] > 0, df['Return']**2, 0)

neg_realized_volatility = df.groupby('Date')['RV_neg'].sum().reset_index()
neg_realized_volatility.columns = ['Date', 'RV_neg']

pos_realized_volatility = df.groupby('Date')['RV_pos'].sum().reset_index()
pos_realized_volatility.columns = ['Date', 'RV_pos']

# Calculate realized quadratic variation (RQ)
df['RQ'] = df['Return']**4
realized_quadratic_variation = df.groupby('Date')['RQ'].sum().reset_index()
realized_quadratic_variation.columns = ['Date', 'RQ']

# Merge all the calculations into one DataFrame
result = pd.merge(realized_volatility, bipower_variation, on='Date')
result = pd.merge(result, neg_realized_volatility, on='Date')
result = pd.merge(result, pos_realized_volatility, on='Date')
result = pd.merge(result, realized_quadratic_variation, on='Date')

# Ensure the Date column is in the format y-m-d
result['Date'] = pd.to_datetime(result['Date']).dt.strftime('%Y-%m-%d')

# Print or save the result
print(result)

            Date        RV       BVP    RV_neg    RV_pos         RQ
0     2009-01-26  0.083050  0.053234  0.010838  0.072212   0.002571
1     2009-01-27  3.757399  2.334166  2.406640  1.350759   0.433292
2     2009-01-28  4.232781  2.615121  2.131254  2.101527   0.525997
3     2009-01-29  2.826337  1.787276  1.303102  1.523234   0.161794
4     2009-01-30  2.201264  1.325969  1.160703  1.040561   0.129156
...          ...       ...       ...       ...       ...        ...
4896  2024-10-24  0.474358  0.257944  0.282357  0.192001   0.005588
4897  2024-10-25  0.319412  0.165003  0.117829  0.201583   0.005298
4898  2024-10-27  4.033761  0.134616  3.942295  0.091466  15.126110
4899  2024-10-28  0.509773  0.321941  0.271659  0.238114   0.005412
4900  2024-10-29  0.421618  0.245423  0.227144  0.194473   0.004051

[4901 rows x 6 columns]


In [9]:
result['Date'] = pd.to_datetime(result['Date'])  # 'errors=coerce' will turn invalid parsing into NaT
data['Date'] = pd.to_datetime(data['Date'])

In [10]:
merged_df = pd.merge(data, result, on='Date', how='inner')
merged_df

,Date,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_4091,embedding_4092,embedding_4093,embedding_4094,embedding_4095,RV,BVP,RV_neg,RV_pos,RQ
0,2014-01-01,-1.517078,-2.217229,-0.913382,-0.507521,-0.611082,0.249111,-2.486910,-0.126034,-3.137839,...,0.881849,0.834024,-0.407182,-0.447372,0.976078,0.006140,0.002674,0.001298,0.004842,0.000006
1,2014-01-02,-0.979689,-2.329813,0.262478,0.485761,1.299325,0.269965,-1.420801,-1.407264,-2.907781,...,1.295898,0.694091,-1.279165,-0.369629,0.759160,0.209711,0.117898,0.159973,0.049738,0.001721
2,2014-01-03,-0.997662,-1.799282,0.674190,0.402943,1.217756,0.230436,-1.553128,-0.922233,-3.123052,...,1.504599,0.861377,-1.231664,-0.024600,0.793679,0.115568,0.069569,0.068341,0.047227,0.000271
3,2014-01-05,-0.737085,-2.182177,-0.897072,0.597660,0.194668,-0.449157,-2.243655,-2.043231,-3.896882,...,1.745685,2.008193,-1.202095,-0.473968,-0.324746,0.013734,0.008607,0.007153,0.006580,0.000019
4,2014-01-06,-1.069120,-2.729595,0.420655,0.334751,1.597477,-0.021764,-1.626255,-0.959567,-3.131338,...,1.248603,0.548811,-1.158162,0.371242,0.573909,0.092181,0.051935,0.050780,0.041400,0.000214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,2024-01-26,-0.667137,-2.011181,0.333467,0.494063,1.752181,0.226858,-1.593692,-1.167683,-3.468695,...,1.031001,1.209613,-0.649497,0.157978,0.585307,0.406610,0.251700,0.180266,0.226343,0.005369
3131,2024-01-28,-0.673623,-1.321869,0.994243,0.413169,1.989308,0.059535,-1.867169,-1.133411,-3.478577,...,1.060372,0.895646,-0.587073,-0.038206,0.236125,0.345671,0.120092,0.094935,0.250736,0.042749
3132,2024-01-29,-0.891116,-2.005443,0.658899,0.488497,1.662750,0.383880,-1.257140,-1.074754,-3.505890,...,1.135375,1.181258,-0.624681,0.177267,0.628000,0.523707,0.294702,0.324536,0.199172,0.007288
3133,2024-01-30,-0.635582,-1.944958,0.485915,0.336782,1.753944,0.361026,-1.635296,-1.359022,-3.067884,...,0.768250,1.004859,-0.593746,0.063804,0.571134,0.412695,0.238419,0.191921,0.220774,0.003199


In [6]:
merged_df = merged_df.drop('Unnamed: 0.1', axis = 1)
merged_df = merged_df.drop('Unnamed: 0', axis = 1)
merged_df

,Date,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_763,feature_764,feature_765,feature_766,feature_767,RV,BVP,RV_neg,RV_pos,RQ
0,2014-01-01,0.061217,0.052544,0.069719,-0.308596,0.105019,0.233055,-0.183720,0.254759,0.296184,...,0.198660,0.231169,0.081846,0.047831,0.259529,0.006140,0.002674,0.001298,0.004842,0.000006
1,2014-01-02,-0.051959,0.062150,0.105734,-0.379149,0.118276,0.192018,-0.208628,0.212018,0.245428,...,0.269643,0.110607,0.081575,-0.141455,0.154967,0.209711,0.117898,0.159973,0.049738,0.001721
2,2014-01-03,-0.027298,0.118546,0.098795,-0.332889,0.105992,0.168794,-0.229695,0.271403,0.242850,...,0.276981,0.113442,0.057419,-0.214177,0.143204,0.115568,0.069569,0.068341,0.047227,0.000271
3,2014-01-05,0.133022,0.050614,0.031066,-0.564844,0.146796,0.168420,-0.245068,0.243157,0.287478,...,0.222534,0.216691,0.169857,-0.150433,0.280540,0.013734,0.008607,0.007153,0.006580,0.000019
4,2014-01-06,-0.021880,0.098035,0.105602,-0.389571,0.082481,0.219261,-0.224083,0.274258,0.273309,...,0.265803,0.130290,0.054891,-0.168316,0.141874,0.092181,0.051935,0.050780,0.041400,0.000214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,2023-12-25,-0.006930,0.146332,-0.154860,0.649874,0.139740,0.320529,0.001381,-0.360797,0.242845,...,0.341624,0.210286,-0.092258,-0.267548,-0.145091,0.062109,0.038685,0.035350,0.026759,0.000686
3104,2023-12-26,0.022700,0.168835,-0.049278,0.558618,0.153206,0.324008,-0.022397,-0.445919,0.195445,...,0.305753,0.185498,-0.064977,-0.276837,-0.191121,0.592945,0.353216,0.280096,0.312849,0.034302
3105,2023-12-27,0.002978,0.131368,-0.071338,0.585990,0.161807,0.330345,-0.005222,-0.430345,0.183310,...,0.324113,0.180822,-0.121262,-0.272223,-0.230341,0.475644,0.249121,0.283001,0.192643,0.013576
3106,2023-12-28,-0.013278,0.121760,-0.059043,0.584970,0.162755,0.281890,-0.011645,-0.422384,0.156002,...,0.301516,0.175790,-0.126492,-0.292001,-0.239805,0.821358,0.431317,0.519285,0.302073,0.026007


In [11]:
merged_df.to_csv('embedding_Llama_mean_Brent.csv')